In [1]:
import sys
import os
import pandas as pd
import logging
from tqdm import tqdm
import statsmodels.api as sm
import datetime
import sqlite3
from scipy import stats
from statsmodels.stats.diagnostic import het_white

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

sys.path.insert(0, r"E:/Thesis/crypto_project")
from factor_model.model_update.loaders.crypto_coin_loaders import (
    get_extended_ticker_list,
)
from factor_model.model_update.loaders.price_data_loaders import (
    generate_ytickets,
    generate_price_data_map,
)
from factor_model.model_update.loaders.market_cap_loader import (
    generate_market_cap_only,
)
from factor_model.model_update.loaders.crypto_coin_loaders import (
    get_extended_ticker_list,
)
from statsmodels.stats.outliers_influence import variance_inflation_factor

from factor_model.model_update.return_generators.total_return_calculation import (
    generate_return_data,
)
from datetime import date, timedelta
from factor_model.model_update.database_generators.generate_databases import (
    refresh_database,
)
from factor_model.model_update.estimation_universe.estimation_core_universe import (
    generate_estimation_basis,
)
from factor_model.model_update.styles.return_based import (
    generate_x_month_price_change,
)
from factor_model.model_update.styles.volume_based import (
    generate_x_month_aggregate_volume,
)
from factor_model.model_update.factor_return_estimation.factor_return_estimation import (
    create_factor_return_data,
)
from factor_model.model_update.loaders.risk_free_rate_loader import (
    get_risk_free_rate_history,
)
from factor_model.model_update.return_generators.excess_return_calculation import (
    generate_excess_returns,
)
from factor_model.risk_calculations.core_universe_portfolio import (
    generate_market_portfolio,
)
from factor_model.risk_calculations.specific_risk import (
    generate_raw_portfolio_specific_risk,
)


DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    SPECIFIC_RISK,
    FACTOR_MODEL_ESTIMATES,
    FIX_SET_OF_HALF_LIFES,
)
from factor_model.model_update import UPDATE_PARAMS, RISK_CALCULATION_PARAMETERS

In [2]:
parameters = UPDATE_PARAMS.copy()
parameters["CORE_UNIV_START"] = parameters["ESTIMATION_DAY"] - timedelta(
    parameters["PRESENT_IN_MARKET"]
)
parameters["GENERATE_DATABASE"] = True
parameters["HORIZON"] = "1y"  # 1y is also fair
parameters["NOBS"] = 3000
parameters["MARKET_CAP_COVERAGE"] = 0.7

In [3]:
tickers = get_extended_ticker_list(
    parameters["NOBS"]
)  # the number of crypto can be adjusted...
# TODO : survival bias should be fixed by adding discontiued crypto currency
tickers = [f for f in tickers if f.endswith("USD")]
tickers = sorted(list(set(tickers)))

In [4]:
yfinance_tickers = generate_ytickets(tickers)
price_data_map, tickers_with_missing_data = generate_price_data_map(
    yfinance_tickers, parameters["HORIZON"]
)

problematic_keys = []
for key in price_data_map.keys():
    if len(price_data_map[key].columns) > 8:
        problematic_keys.append(key)

for key in problematic_keys:
    del price_data_map[key]

# refresh tickers and throw out those ones, which have no price history
drop_keys = list()
for key in price_data_map.keys():
    if len(price_data_map[key]) < 2:  # minimum number of obs for returns are 2
        del yfinance_tickers[key]
        drop_keys.append(key)
for key in drop_keys:
    del price_data_map[key]

2024-11-23 10:14:10,343 - ERROR - AIC-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
2024-11-23 10:14:10,346 - WARNING - No data for AIC-USD!
2024-11-23 10:14:14,076 - ERROR - AMC31871-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-23 10:14:14,079 - WARNING - No data for AMC31871-USD!
2024-11-23 10:14:15,585 - ERROR - ANYONE-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-23 10:14:15,590 - WARNING - No data for ANYONE-USD!
2024-11-23 10:14:18,827 - ERROR - ARTFI-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-23 10:14:18,833 - WARNING - No data for ARTFI-USD!
2024-11-23 10:14:20,293 - ERROR - ATH30083-USD: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
2024-11-23 10:14:20,297 - WARNING - No data for ATH30083-USD!
2024-11-23 10:14:22,845 - ERROR - AXLFRXET

In [5]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RAW_DATA_DB)) as conn:
    max_dates = pd.read_sql(
        "SELECT MAX(date) as date, symbol FROM raw_price_data group by symbol", conn
    )
max_dates_struct = max_dates.set_index("symbol").to_dict()
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RAW_DATA_DB)) as conn:
    max_id = pd.read_sql("SELECT MAX(id) as id FROM raw_price_data", conn)
idx_index = int(max_id["id"].values[0]) + 1

In [6]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RAW_DATA_DB)) as conn:
    for key in price_data_map:
        latest_date = max_dates_struct["date"].get(key, "2008-01-01")

        df_temp = price_data_map[key]
        df_temp = df_temp[df_temp["date"].astype(str) > latest_date].copy()
        df_temp["symbol"] = key
        df_temp.columns = [col.lower().replace(" ", "_") for col in df_temp.columns]
        df_temp.reset_index(inplace=True)
        df_temp.rename(columns={"index": "id"}, inplace=True)
        df_temp["id"] = range(idx_index, len(df_temp) + idx_index)
        idx_index += len(df_temp)
        df_temp.to_sql("raw_price_data", conn, if_exists="append", index=False)

In [7]:
# market_cap_df_full = generate_market_cap_data(yfinance_tickers)
market_cap_df = generate_market_cap_only(yfinance_tickers)
# generate square root of cap weighting
market_cap_df["transformed_market_cap"] = parameters["WEIGHT_FUNCTION"](
    market_cap_df["market_cap"]
)
market_cap_df.sort_values(by="transformed_market_cap", ascending=False, inplace=True)
total_return_data_map = generate_return_data(price_data_map)

2024-11-23 10:26:01,725 - INFO - No data for ticker: 2049-USD
2024-11-23 10:26:14,171 - INFO - No data for ticker: ABI30882-USD
2024-11-23 10:26:56,945 - INFO - No data for ticker: AI14838-USD
2024-11-23 10:27:51,828 - INFO - No data for ticker: AMPLE23682-USD
2024-11-23 10:28:26,613 - INFO - No data for ticker: ARC13933-USD
2024-11-23 10:28:47,979 - INFO - No data for ticker: ASEED-USD
2024-11-23 10:29:44,486 - INFO - No data for ticker: BABY21709-USD
2024-11-23 10:30:23,310 - INFO - No data for ticker: BCLAT-USD
2024-11-23 10:30:27,754 - INFO - No data for ticker: BCT24348-USD
2024-11-23 10:31:40,892 - INFO - No data for ticker: BMAX-USD
2024-11-23 10:32:14,623 - INFO - No data for ticker: BOYSCLUB30914-USD
2024-11-23 10:32:19,090 - INFO - No data for ticker: BRCP-USD
2024-11-23 10:32:20,205 - INFO - No data for ticker: BRCT-USD
2024-11-23 10:34:58,718 - INFO - No data for ticker: CNG29188-USD
2024-11-23 10:36:24,067 - INFO - No data for ticker: CUFF-USD
2024-11-23 10:36:34,156 - INF

ReadTimeout: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=30)

In [ ]:
estimation_dates = list(
    total_return_data_map["BTC-USD"].tail(parameters["ESTIMATION_HORIZON"])["date"]
)
estimation_dates = list(
    total_return_data_map["ETC-USD"].tail(parameters["ESTIMATION_HORIZON"])["date"]
)

with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    max_fac_return_date = pd.read_sql(
        "SELECT MAX(date) as date FROM factor_returns", conn
    )["date"].values[0]
last_estima_date = date(
    int(max_fac_return_date[0:4]),
    int(max_fac_return_date[5:7]),
    int(max_fac_return_date[8:]),
)
estimation_dates = [
    date_value
    for date_value in estimation_dates
    if date_value > datetime.date(2018, 5, 8) and date_value > last_estima_date
]  # six months after ETH-USD has data in yfinance

In [ ]:
estimation_dates

[datetime.date(2024, 11, 20), datetime.date(2024, 11, 21)]

In [ ]:
risk_free_rate_data = get_risk_free_rate_history(estimation_dates)
excess_return_data_map = generate_excess_returns(
    total_return_data_map, risk_free_rate_data
)

[*********************100%***********************]  1 of 1 completed
2024-11-21 14:47:49,752 - INFO - First SOFR date is 2024-11-11 00:00:00, before this date, the 3M US Treasury rate is used as risk free rate.
2024-11-21 14:47:49,756 - INFO - 3M US Treasury rate is used for 4994 trading days.
E:\Thesis/crypto_project\factor_model\model_update\loaders\risk_free_rate_loader.py:63: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ].fillna(method="backfill")
E:\Thesis/crypto_project\factor_model\model_update\loaders\risk_free_rate_loader.py:77: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  treasury_data["risk_free_rate"] = treasury_data["risk_free_rate"].fillna(
E:\Thesis/crypto_project\factor_model\model_update\loaders\risk_free_rate_loader.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame 

In [ ]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    max_dates = pd.read_sql(
        "SELECT MAX(date) as date, symbol FROM returns group by symbol", conn
    )
max_dates_struct = max_dates.set_index("symbol").to_dict()

In [ ]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    for key in excess_return_data_map:
        latest_date = max_dates_struct["date"].get(key, "2008-01-01")
        df_temp = excess_return_data_map[key]
        df_temp = df_temp[df_temp["date"].astype(str) > latest_date].copy()
        df_temp["symbol"] = key
        df_temp.columns = [col.lower().replace(" ", "_") for col in df_temp.columns]
        df_temp.reset_index(inplace=True)
        df_temp.rename(columns={"index": "id"}, inplace=True)
        df_temp["id"] = range(idx_index, len(df_temp) + idx_index)
        idx_index += len(df_temp)
        df_temp.to_sql("returns", conn, if_exists="append", index=False)

In [ ]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    max_date_risk_free = pd.read_sql(
        "SELECT MAX(date) as date FROM risk_free_rates", conn
    )["date"].values[0]
    last_estima_date = date(
        int(max_date_risk_free[0:4]),
        int(max_date_risk_free[5:7]),
        int(max_date_risk_free[8:]),
    )
    rft_sel = risk_free_rate_data[risk_free_rate_data["date"] > last_estima_date].copy()
    rft_sel["id"] = range(0, len(rft_sel))
    rft_sel.to_sql("risk_free_rates", conn, if_exists="append", index=False)

In [ ]:
momentum_move_map = generate_x_month_price_change(
    price_data_map, x_len=6, month_len=parameters["MONTH_LENGTH"]
)
reversal_map = generate_x_month_price_change(
    price_data_map, x_len=1, month_len=parameters["MONTH_LENGTH"]
)
volume_map = generate_x_month_aggregate_volume(
    price_data_map, x_len=1, month_len=parameters["MONTH_LENGTH"]
)
daily_data_maps = {}
daily_data_maps["reversal"] = reversal_map
daily_data_maps["momentum"] = momentum_move_map
daily_data_maps["return"] = excess_return_data_map
daily_data_maps["volume"] = volume_map

E:\Thesis/crypto_project\factor_model\model_update\styles\return_based.py:21: RuntimeWarning: divide by zero encountered in divide
  close_price_changes = np.matrix(
E:\Thesis/crypto_project\factor_model\model_update\styles\return_based.py:21: RuntimeWarning: invalid value encountered in divide
  close_price_changes = np.matrix(


In [ ]:
for key in excess_return_data_map.keys():
    try:
        v = min(excess_return_data_map[key]["date"])
    except Exception:
        print(key)

In [ ]:
tstats_all = list()
coefficients_all = list()
vif_all = list()
model_summary = list()

for date in tqdm(estimation_dates):
    # step 0 : assemble and save estimation data
    estimation_basis, _ = generate_estimation_basis(
        excess_return_data_map, market_cap_df, date, parameters
    )
    factor_return_data = create_factor_return_data(
        estimation_basis, parameters, date, daily_data_maps
    )
    factor_return_data_core = factor_return_data[
        factor_return_data["core_universe"] > 0
    ]
    if parameters["GENERATE_DATABASE"]:
        refresh_database(
            symbol_level_data={
                str(date): factor_return_data.rename(columns={"ticker": "symbol"})
            },
            database_location=DATABASE_LOCATION,
            database_name=FACTOR_MODEL_ESTIMATES,
            database_table_name="exposures",
            delete_database=False,
            key_field_name="date",
            update_mode="append",
        )
    # step 1 estimation
    try:
        mod_wls = sm.WLS(
            endog=factor_return_data_core["return"],
            exog=factor_return_data_core[parameters["REGRESSORS_SET1"]],
            weights=factor_return_data_core["transformed_market_cap"],
        ).fit()

        # step 2 save results
        coefficient_date = mod_wls.params.to_frame().T
        coefficient_date["date"] = date
        coefficients_all.append(coefficient_date)

        tstat_date = mod_wls.tvalues.to_frame().T
        tstat_date["date"] = date
        tstats_all.append(tstat_date)

        # step 2 b, additional tests...
        white_test = het_white(mod_wls.resid,  mod_wls.model.exog)
        res_jb = stats.jarque_bera(mod_wls.resid)

        model_summary.append(
            pd.DataFrame(
                {
                    "date": [date],
                    "r2_core": [mod_wls.rsquared],
                    "r2_adj": [mod_wls.rsquared_adj],
                    "nobs": [mod_wls.nobs],
                    "white_lm_pvalue": [float(white_test[1])],
                    "jb_pvalue": [float(res_jb.pvalue)]
                }
            )
        )

        exog = factor_return_data[
            parameters["REGRESSORS_SET1"] + parameters["REGRESSORS_SET2"]
        ].to_numpy()
        num_features = exog.shape[1]
        vif_values = [variance_inflation_factor(exog, i) for i in range(num_features)]
        vif_result_date = (
            pd.Series(
                vif_values,
                index=parameters["REGRESSORS_SET1"] + parameters["REGRESSORS_SET2"],
            )
            .to_frame()
            .T
        )
        vif_result_date["date"] = date
        vif_all.append(vif_result_date)

        specific_returns = factor_return_data[["ticker"]].copy()
        specific_returns["specific_return"] = factor_return_data[
            "return"
        ] - mod_wls.predict(factor_return_data[parameters["REGRESSORS_SET1"]])
        if parameters["GENERATE_DATABASE"]:
            refresh_database(
                symbol_level_data={
                    str(date): specific_returns.rename(columns={"ticker": "symbol"})
                },
                database_location=DATABASE_LOCATION,
                database_name=FACTOR_MODEL_ESTIMATES,
                database_table_name="specific_returns",
                delete_database=False,
                key_field_name="date",
                update_mode="append",
            )
    except Exception:
        print(date)

100%|██████████| 2/2 [00:12<00:00,  6.01s/it]


In [ ]:
all_tstat = pd.concat(tstats_all, axis=0)
all_coeff = pd.concat(coefficients_all, axis=0)
all_vif = pd.concat(vif_all, axis=0)
all_model_summary = pd.concat(model_summary, axis=0)
if parameters["GENERATE_DATABASE"]:
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_tstat},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="t_statistics",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_coeff},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="factor_returns",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_vif},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="vifs",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )
    refresh_database(
        symbol_level_data={str(datetime.date.today()): all_model_summary},
        database_location=DATABASE_LOCATION,
        database_name=FACTOR_MODEL_ESTIMATES,
        database_table_name="r_squares",
        delete_database=False,
        key_field_name="version_date",
        update_mode="append",
        drop_id_col=True,
    )

In [ ]:
# specific risk related

In [ ]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    full_specific_returns = pd.read_sql_query(f"SELECT * FROM specific_returns", conn)

with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    estim_universe = pd.read_sql_query(
        f"SELECT symbol, transformed_market_cap, date, core_universe FROM exposures where core_universe > 0",
        conn,
    )

In [ ]:
full_specific_returns = full_specific_returns[
    full_specific_returns["symbol"].isin(
        list(set(estim_universe[estim_universe["core_universe"] > 0]["symbol"]))
    )
].copy()


In [ ]:
dates = [
    date_v
    for date_v in list(full_specific_returns["date"].unique())[1:]
    if date_v in [str(d) for d in estimation_dates]
]
risk_calculation_parameters = RISK_CALCULATION_PARAMETERS.copy()

In [ ]:
def multi_process_calc(temp_half_life):
    risk_calculation_parameters = {"specific_risk_half_life": temp_half_life}
    risk_for_half_life = full_specific_returns.groupby("symbol").apply(
        lambda group: pd.DataFrame({
            'date': group['date'],
            'specific_risk': group['specific_return']
                .ewm(halflife=risk_calculation_parameters['specific_risk_half_life'])
                .std(),
            'symbol': group['symbol'].iloc[0]
        })
    )
    return risk_for_half_life.reset_index(drop=True)

In [ ]:
all_core_spec_risk = {}

for temp_half_life in tqdm(FIX_SET_OF_HALF_LIFES):
    all_core_spec_risk[temp_half_life] = multi_process_calc(temp_half_life)
    all_core_spec_risk[temp_half_life].set_index(["date", "symbol"], inplace=True)

  0%|          | 0/38 [00:00<?, ?it/s]C:\Users\Bebesi Laci\AppData\Local\Temp\ipykernel_22648\4244655963.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  risk_for_half_life = full_specific_returns.groupby("symbol").apply(
  3%|▎         | 1/38 [00:05<03:29,  5.66s/it]C:\Users\Bebesi Laci\AppData\Local\Temp\ipykernel_22648\4244655963.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  risk_for_half_life =

In [ ]:
estim_universe_dict = {date: df for date, df in estim_universe.groupby("date")}
for key in all_core_spec_risk.keys():
    all_core_spec_risk[key] = {
        date: df for date, df in all_core_spec_risk[key].groupby("date")
    }

In [ ]:
all_combined_specific_risk_estimates = list()

for date in tqdm(dates):
    # 1. gather exposures
    exposure = estim_universe_dict[date]
    # 2. identify universe_info
    market_portfolio = generate_market_portfolio(exposure)

    for half_life in FIX_SET_OF_HALF_LIFES:
        # 3. grab relevant spec risks
        tempd_df = all_core_spec_risk[half_life][date]
        tempd_df.reset_index(inplace=True)
        tempd_df.set_index("symbol", inplace=True)
        tempd_df.dropna(inplace=True)

        all_combined_specific_risk_estimates.append(
            pd.DataFrame(
                {
                    "date": [date],
                    "half_life": [half_life],
                    "specific_risk": generate_raw_portfolio_specific_risk(
                        tempd_df.to_dict()["specific_risk"], market_portfolio
                    ),
                }
            )
        )

100%|██████████| 2/2 [00:00<00:00,  4.17it/s]


In [ ]:
combined_spec_risk = pd.concat(all_combined_specific_risk_estimates)


In [ ]:
refresh_database(
    symbol_level_data={str(date): combined_spec_risk},
    database_location=DATABASE_LOCATION,
    database_name=SPECIFIC_RISK,
    database_table_name="core_specific_risk",
    delete_database=False,
    key_field_name="version_date",
    update_mode="append",
)

In [ ]:
all_model_summary

,date,r2_core,r2_adj,nobs,white_lm_pvalue,jb_pvalue
0,2024-11-20,0.221149,0.208505,314.0,0.451407,0.0
0,2024-11-21,0.085118,0.070168,312.0,0.007796,0.0


In [ ]:
all_tstat

,market,size,momentum,reversal,volume,new_coin,date
0,-9.993482,3.828505,0.034661,1.570864,0.458065,-2.380895,2024-11-20
0,21.202661,0.563903,-0.905451,-4.251141,-0.347137,1.208347,2024-11-21


In [ ]:
all_vif

,market,size,momentum,reversal,volume,new_coin,date
0,41.984476,1.048251,1.350790,1.325009,1.023880,1.006443,2024-11-20
0,40.935562,1.047631,1.261627,1.238872,1.024435,1.006411,2024-11-21
